Read .nc files

In [1]:
import xarray as xr

# Open the NetCDF file
file_path = r"D:\IPMA\Results\Processed\CO_meteo_global_mean.nc"
ds = xr.open_dataset(file_path)

# Display the dataset's structure and metadata
print("Dataset Overview:\n")
print(ds)

# List all variables in the dataset
print("\nAvailable Variables:")
for var in ds.variables:
    print(f"- {var}: {ds[var].dims}, {ds[var].shape}")


Dataset Overview:

<xarray.Dataset> Size: 68kB
Dimensions:                     (latitude: 43, longitude: 65)
Coordinates:
  * latitude                    (latitude) float64 344B 34.5 35.25 ... 66.0
  * longitude                   (longitude) float64 520B -12.0 -11.25 ... 36.0
Data variables:
    precip_Total_Precipitation  (latitude, longitude) float64 22kB ...
    temp_Max                    (latitude, longitude) float64 22kB ...
    wind_Max                    (latitude, longitude) float64 22kB ...

Available Variables:
- latitude: ('latitude',), (43,)
- longitude: ('longitude',), (65,)
- precip_Total_Precipitation: ('latitude', 'longitude'), (43, 65)
- temp_Max: ('latitude', 'longitude'), (43, 65)
- wind_Max: ('latitude', 'longitude'), (43, 65)


View nc file in table

In [1]:
import xarray as xr
import pandas as pd

# Open the NetCDF file
nc_file = r"D:\IPMA\Results\co_fire_meteo_Iberia.nc"
ds = xr.open_dataset(nc_file)

# Convert all variables to a DataFrame
df = ds.to_dataframe().reset_index()

# Display the first few rows
print(df.head())

df


KeyboardInterrupt: 